In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import utils.read_data as rd
import utils.preprocessing as pp
import utils.io_model as im
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path
import tensorflow as tf

In [2]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/'

# ジャパンカップ当日の別レースデータをこのノートブックで使えるようにディレクトリに配置
import shutil

target_file_names = [name for name in os.listdir(DATA_PATH+"test_data") if not "japan" in name and  not "arima" in name]
race_files = sorted([name for name in target_file_names if "race" in name])
horse_files = sorted([name for name in target_file_names if "horse" in name])
race_names = [race_name.split("-")[1] for race_name in race_files]
race_names = [race_name.split(".")[0] for race_name in race_files]
for race_name, race_file, horse_file in zip(race_names, race_files, horse_files):
    target_data_path = DATA_PATH+"test_data/target_"+race_name
    os.makedirs(target_data_path, exist_ok=True)
    shutil.move(DATA_PATH+"test_data/"+race_file, target_data_path+"/")
    shutil.move(DATA_PATH+"test_data/"+horse_file, target_data_path+"/")

In [3]:
os.listdir(DATA_PATH+"test_data")

['.ipynb_checkpoints',
 'arima_horse.csv',
 'arima_race.csv',
 'japancup_horse.csv',
 'japancup_race.csv',
 'target_race-202005050901',
 'target_race-202005050902',
 'target_race-202005050903',
 'target_race-202005050904',
 'target_race-202005050905',
 'target_race-202005050906',
 'target_race-202005050907',
 'target_race-202005050908',
 'target_race-202005050909',
 'target_race-202005050910',
 'target_race-202005050911',
 'target_race-202009050901',
 'target_race-202009050902',
 'target_race-202009050903',
 'target_race-202009050904',
 'target_race-202009050905',
 'target_race-202009050906',
 'target_race-202009050907',
 'target_race-202009050908',
 'target_race-202009050909',
 'target_race-202009050910',
 'target_race-202009050911',
 'target_race-202009050912']

In [17]:
TARGET_DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/test_data/target_race-202005050902/'

# 前処理

In [18]:
df_target_horse = rd.read_target_horse_csv(TARGET_DATA_PATH)
df_target_race  = rd.read_target_race_csv(TARGET_DATA_PATH)
df_target = pd.merge(df_target_horse, df_target_race, on='race_id', how='left')

In [19]:
df_horse = rd.read_horse_csv(DATA_PATH)
df_race = rd.read_race_csv(DATA_PATH)
#df = pd.read_csv(DATA_PATH+"/learning/horse_race.csv")
df = pd.merge(df_horse, df_race, on='race_id', how='left')

c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [20]:
df = pp.join_n_race_for_prediction(df_target, df, 3)

In [21]:
df.head()

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,...,hukusyo_second-3,hukusyo_third-3,wakuren-3,umaren-3,wide_1_2-3,wide_1_3-3,wide_2_3-3,umatan-3,renhuku3-3,rentan3-3
0,2.020051e+11,1,3,6,2018105194,牡2,55,5339,1:35.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.020051e+11,2,8,16,2018101300,牡2,55,1174,1:35.4,3/4,...,160,150,"10,260","9,980","1,900","2,020",350,"28,680","9,300","121,720"
2,2.020051e+11,3,8,18,2018102169,牡2,52,1184,1:35.6,1.1/4,...,130,980,610,840,420,"5,450","2,070","1,820","13,100","52,610"
3,2.020051e+11,4,2,4,2018103602,牡2,55,1169,1:35.8,1.1/2,...,130,980,610,840,420,"5,450","2,070","1,820","13,100","52,610"
4,2.020051e+11,5,6,11,2018103916,牡2,55,5386,1:35.9,クビ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 前処理

In [22]:
columns = ["race_course", "weather", "ground_status", 
           "where_racecourse", "race_class", "running_condition", 
           "frame_number", "horse_number",
           "sex_and_age", "burden_weight", "rider_id", 
           "tamer_id", "horse_weight",
           "total_horse_number_x", 
           "rank-1", "rank-2", "rank-3",
           "total_horse_number_x-1", "total_horse_number_x-2","total_horse_number_x-3",
           "goal_time-1", "goal_time-2", "goal_time-3",
           "last_time-1", "last_time-2", "last_time-3", 
           "half_way_rank-1", "half_way_rank-2", "half_way_rank-3", 
           "prize-1", "prize-2", "prize-3"]

df = df[columns]

In [23]:
df["where_racecourse"] = df["where_racecourse"].map(pp.extract_place)

df["sex"] = df["sex_and_age"].map(lambda sex_and_age: sex_and_age[0])
df["age"] = df["sex_and_age"].map(lambda sex_and_age: sex_and_age[1:])

df["goal_time-1"] = df["goal_time-1"].map(pp.to_seconds)
df["goal_time-2"] = df["goal_time-2"].map(pp.to_seconds)
df["goal_time-3"] = df["goal_time-3"].map(pp.to_seconds)

df["horse_weight"] = df["horse_weight"].map(pp.extract_weight).astype(np.int64)

df["prize-1"] = df["prize-1"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df["prize-2"] = df["prize-2"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df["prize-3"] = df["prize-3"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)

df["kyakusitu-1"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-1"].values, df["half_way_rank-1"])]
df["kyakusitu-2"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-2"].values, df["half_way_rank-2"])]
df["kyakusitu-3"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-3"].values, df["half_way_rank-3"])]

# 欠損値処理
df = df.replace('---', -1)
df = df.fillna(-1)

In [24]:
df["rank-1"] = pp.make_label(df["rank-1"].values, df["total_horse_number_x-1"].values)
df["rank-2"] = pp.make_label(df["rank-2"].values, df["total_horse_number_x-2"].values)
df["rank-3"] = pp.make_label(df["rank-3"].values, df["total_horse_number_x-3"].values)

In [25]:
def fill_missing_columns(df_a, df_b):
    columns_for_b = set(df_a.columns) - set(df_b.columns)
    for column in columns_for_b:
        df_b[column] = 0

    columns_for_a = set(df_b.columns) - set(df_a.columns)
    for column in columns_for_a:
        df_a[column] = 0

In [26]:
# targetデータを予測に用いる為に、targetデータに存在していない学習時に用いたカテゴリデータを追加
df_used_learning = pd.read_csv(DATA_PATH+"/learning/horse_race.csv")

columns = ["race_course", "weather", "ground_status", 
           "where_racecourse", "race_class", "running_condition", 
           "frame_number", "horse_number",
           "sex_and_age", "burden_weight", "rider_id", 
           "tamer_id", "horse_weight",
           "total_horse_number_x", 
           "rank-1", "rank-2", "rank-3",
           "total_horse_number_x-1", "total_horse_number_x-2","total_horse_number_x-3",
           "goal_time-1", "goal_time-2", "goal_time-3",
           "last_time-1", "last_time-2", "last_time-3", 
           "half_way_rank-1", "half_way_rank-2", "half_way_rank-3", 
           "prize-1", "prize-2", "prize-3"]

df_used_learning = df_used_learning[columns]

df_used_learning["where_racecourse"] = df_used_learning["where_racecourse"].map(pp.extract_place)

df_used_learning["sex"] = df_used_learning["sex_and_age"].map(lambda sex_and_age: sex_and_age[0])
df_used_learning["age"] = df_used_learning["sex_and_age"].map(lambda sex_and_age: sex_and_age[1:])

df_used_learning["goal_time-1"] = df_used_learning["goal_time-1"].map(pp.to_seconds)
df_used_learning["goal_time-2"] = df_used_learning["goal_time-2"].map(pp.to_seconds)
df_used_learning["goal_time-3"] = df_used_learning["goal_time-3"].map(pp.to_seconds)

df_used_learning["horse_weight"] = df_used_learning["horse_weight"].map(pp.extract_weight).astype(np.int64)

df_used_learning["prize-1"] = df_used_learning["prize-1"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df_used_learning["prize-2"] = df_used_learning["prize-2"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df_used_learning["prize-3"] = df_used_learning["prize-3"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)

df_used_learning["kyakusitu-1"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df_used_learning["total_horse_number_x-1"].values, df_used_learning["half_way_rank-1"])]
df_used_learning["kyakusitu-2"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df_used_learning["total_horse_number_x-2"].values, df_used_learning["half_way_rank-2"])]
df_used_learning["kyakusitu-3"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df_used_learning["total_horse_number_x-3"].values, df_used_learning["half_way_rank-3"])]

# 欠損値処理
df_used_learning = df_used_learning.replace('---', -1)
df_used_learning = df_used_learning.fillna(-1)

df_used_learning["rank-1"] = pp.make_label(df_used_learning["rank-1"].values, df_used_learning["total_horse_number_x-1"].values)
df_used_learning["rank-2"] = pp.make_label(df_used_learning["rank-2"].values, df_used_learning["total_horse_number_x-2"].values)
df_used_learning["rank-3"] = pp.make_label(df_used_learning["rank-3"].values, df_used_learning["total_horse_number_x-3"].values)

input_columns = ["race_course", "weather", "ground_status", 
                 "where_racecourse", "race_class", "running_condition", 
                 "frame_number", "horse_number",
                 "sex", "age", "burden_weight", "rider_id", 
                 "tamer_id", "horse_weight",
                 "rank-1", "rank-2", "rank-3", 
                 "goal_time-1", "goal_time-2", "goal_time-3",
                 "last_time-1", "last_time-2", "last_time-3", 
                 "kyakusitu-1", "kyakusitu-2", "kyakusitu-3", 
                 "prize-1", "prize-2", "prize-3"]

df_used_learning = pp.one_hot_encoding(df_used_learning[input_columns])
df = pp.one_hot_encoding(df[input_columns])

fill_missing_columns(df, df_used_learning)

c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (13,17,50,61,65,98,109,113,146,157,161,162,194,205,209,210,230,242,253,257,258,278,290,301,305,306,326,338,342,345,346,347,349,351,353,354,356,357,358,359,360,361,362,363,364,365,373,374,375,376,377,378,379,380,381,382,383,384,386,390,393,394,395,397,399,401,402,404,405,406,407,408,409,410,411,412,413,421,422,423,424,425,426,427,428,429,430,431,432,434,438,441,442,443,445,447,449,450,452,453,454,455,456,457,458,459,460,461,469,470,471,472,473,474,475,476,477,478,479,480,482,486,489,490,491,493,495,498,500,501,502,503,504,505,506,507,508,509,517,518,519,520,521,522,523,524,525,526,527,528,530,534,537,538,539,541,543,546,548,549,550,551,552,553,554,555,556,557,565,566,567,568,569,570,571,572,573,574,575,576,578,582,585,586,587,589,591,594,596,597,598,599,600,601,602,603,604,605,613,614,615,616,617,618,619,620,621,622,623,624,626,630,633,634,635,637,639,642,644,645,646,647,648

# 予測

In [27]:
model = im.read_model("second_model")

second_model is read.


In [28]:
x = np.array(df)

In [29]:
test = []
for x_input, umaban, horse_id in zip(x, list(range(len(df_target["horse_id"]))), list(df_target["horse_id"])):
    pred = model.predict(x_input.reshape(1, -1))
    if umaban == 1 or umaban == 2 or umaban == 4 or umaban == 5:
        test.append(x_input)
    prediction_class = np.argmax([pred[0][0], pred[0][1], pred[0][2]])
    print("class: %d, (%f, %f, %f), %d (%d)" % (prediction_class, pred[0][0], pred[0][1], pred[0][2], umaban + 1, horse_id))

class: 0, (0.435567, 0.306995, 0.257438), 1 (2018105194)
class: 1, (0.218176, 0.396870, 0.384954), 2 (2018101300)
class: 0, (0.417307, 0.354684, 0.228009), 3 (2018102169)
class: 0, (0.645706, 0.261605, 0.092689), 4 (2018103602)
class: 0, (0.435567, 0.306995, 0.257438), 5 (2018103916)
class: 0, (0.385857, 0.367938, 0.246205), 6 (2018101539)
class: 2, (0.239870, 0.366521, 0.393608), 7 (2018103506)
class: 0, (0.413856, 0.369889, 0.216255), 8 (2018100910)
class: 2, (0.300745, 0.324227, 0.375028), 9 (2018103543)
class: 0, (0.572730, 0.294044, 0.133226), 10 (2018105817)
class: 0, (0.468444, 0.347643, 0.183912), 11 (2018102186)
class: 2, (0.291840, 0.341069, 0.367091), 12 (2018102982)
class: 0, (0.397020, 0.359562, 0.243417), 13 (2018110035)
class: 2, (0.209530, 0.311287, 0.479183), 14 (2018104451)
class: 2, (0.228788, 0.361689, 0.409523), 15 (2018102664)
class: 2, (0.168727, 0.407131, 0.424142), 16 (2018102675)
class: 2, (0.297681, 0.349300, 0.353019), 17 (2018103557)
class: 2, (0.220202, 0.